<a href="https://colab.research.google.com/github/mandemadhura/crewai_exercises/blob/main/explore_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
from crewai import Agent, Task, Crew

In [23]:
import os
# set OPENI KEY
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"
# set SERPER KEY

In [36]:
investment_agent = Agent(
    role="Investment Advisor",
    goal="Design best investment plan that satisfy customer need",
    backstory=(
        "As a part of the dynamic team at a leading investment organization, "
        "your mission is to scour "
        "the investment landscape for potential {client} in {country}. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is to understand clients wish to invest in {investment_type} "
        "based on his {income}, {country} the client is residing in "
        "and the {percentage} of it he/she wish to invest."
    ),
    allow_delegation=False,
    verbose=True
)

In [37]:
lead_investment_agent = Agent(
    role="Portfolio Manager",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [38]:
from crewai_tools import FileReadTool, \
                         SerperDevTool

In [39]:
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [40]:
from crewai_tools import BaseTool
class SentimentAnalysis(BaseTool):
  name: str = "Sentiment Analysis Tool"
  description: str = "Analyses sentiment of the text."

  def _run():
    return "Positive"

In [41]:
sentiment_anlysis_tool = SentimentAnalysis()

In [53]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of a {investment_type}, "
        "in which the client {client} "
        "that recently showed interest to invest. "
        "Utilize all available data sources "
        "to compile the information, "
        "focusing on long term investment plan. Understand the income vs "
        "percentage value to be invested. Come up with best suitable investment "
        "that align with this need. "
        "Study the {investment_type} and the company who offers "
        "that investment throughly based on their financial results so far."
        "Dont rely on only one provider, give combination of best suitable providers "
        "to start investing for.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {investment_type}, "
        "highlighting the client's need and the best personalized "
        "long term investment plan in {country} which can assure the client and "
        "client's family from future financial burden."
    ),
    tools=[file_read_tool, search_tool],
    agent=investment_agent,
)

In [54]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {investment_type}, "
        "craft a personalized outreach email "
        "aimed at best possible long term investment plan that can "
        "secure client's future financially in {country} with multiple providers possible. "
        "Try to ask for a meeting or a brief call to explain this.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {client}, "
        "specifically targeting {investment_type}."
        "Each draft should include "
        "a compelling narrative that explains best investment "
        "plan for their future goals. "
        "Ensure the tone is engaging, professional."
    ),
    tools=[sentiment_anlysis_tool, search_tool],
    agent=lead_investment_agent,
)

In [55]:
crew = Crew(
    agents=[investment_agent,
            lead_investment_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=True
)

In [58]:
inputs = {
    "client": "Madhura Vora",
    "investment_type": "Mutual Fund",
    "income": "4000000",
    "percentage": "20",
    "country": "India"
}

result = crew.kickoff(inputs=inputs)

In [59]:
from IPython.display import Markdown
Markdown(result)

Subject: Personalized Investment Plan Tailored for Your Financial Future

Dear Madhura Vora,

I hope this email finds you well. After conducting thorough research on Mutual Funds in India, I have come across some exciting investment opportunities that align with your long-term financial goals and preferences.

Based on your investment amount of INR 4,000,000, I recommend considering a diversified portfolio that includes the following top-performing funds:

1. ICICI Prudential Infrastructure Fund: This fund focuses on infrastructure-related sectors and has shown strong historical performance, making it a suitable option for long-term growth.

2. Motilal Oswal Midcap Fund: With a focus on mid-cap companies, this fund offers the potential for high returns while managing risk effectively.

3. SBI PSU Fund: Investing in public sector undertakings, this fund provides stability and growth opportunities in the current market scenario.

4. Tata Infrastructure Fund: This fund's thematic approach towards infrastructure investments can complement your portfolio and enhance diversification.

5. HDFC Mid Cap Opportunities Direct Plan: By targeting mid-cap opportunities, this fund aims to deliver consistent returns over the long term.

It's important to note that these recommendations are tailored to your financial goals and risk tolerance, which we can discuss further in detail. I would like to invite you for a brief call or meeting to explain how this investment plan can secure your financial future and help you achieve your objectives.

Please let me know a convenient time for you to discuss this personalized investment plan further. I look forward to the opportunity to assist you in making informed investment decisions.

Thank you for considering these recommendations, Madhura. I am here to support you in every step of your investment journey.

Warm regards,

[Your Name]
Portfolio Manager